In [4]:
import igl
from meshplot import plot
import numpy as np
import networkx as nx

# TODO find out if one face is adjacent to another based on shared vertices 
# def is_adjacent (current_face, face):
    # use np.intersect1d(current_face, face) 
    # if two or more vertices are shared the face is adjacent

# TODO find out if two normals point in the same direction 
# def is_normal_same (normal_A, normal_B): 
    # if cross product is zero then normal point in same (or opposite direction)
    # check if result is zero with EPSILON tolerance 
    # use np.cross and np.linalg.norm

# TODO create polyhedral mesh from triangulated mesh by combining adjacent triangular faces that have the same normal
# into polygonal faces
def build_polyhedral_mesh(faces, face_normals):
    # keep track of faces that have been visited and merged
    visited_faces = np.zeros(len(faces))
    polygonal_faces = []
    polygonal_face_normals = []
    # loop through all triangular faces of the mesh
    for face_id, face in enumerate(faces):
        # avoid faces that have been previously visited and merged
        if visited_faces[face_id] == 0:
            # get the normal of the current triangle face (from igl) and add it to the list of normals for polygonal faces
            normal = face_normals[face_id]
            polygonal_face_normals.append(list(normal))
            
#-----------TODOs start here:---------------------
            
            # TODO set the current face as visited
            # adjacent_face_ids = TODO find the list of faces ids that meet the three criteria: 
            # is adjacent to "face", normals point in the same direction and not already visited
            
            # while len(adjacent_face_ids) > 0: # repeat until all adjacent faces with same normals have been visited and merged
                # for adjacent_face_id in adjacent_face_ids: # loop thorugh all faces that meet the criteria for a merge
                    # if visited_faces[adjacent_face_id] != 1: # ignore previously visited faces
                        
                        # TODO update the visited_faces array
                        # TODO get the vertex array of the adjacent face from "faces"
                        
                        # face = merge_triangles(face, adjacent_face) # merges the adjacent face and update the polygonal face
                
                # adjacent_face_ids = TODO update the while loop condition by again by:
                # finding the list of faces ids that meet the three criteria: 
                # is adjacent to "face", normals point in the same direction and not already visited
                
            # adds the completed polygonal face to the polygonal face list
            polygonal_faces.append(face)
    return polygonal_faces, polygonal_face_normals


# helper function:
# merges two faces together. "face" is the polygonal face that will be merged into, while "adjacent_face" is the face to be merged. 
# both are arrays of vertices
def merge_triangles(face, adjacent_face):
    # convert to list for easier handling
    result_face = list(face)
    # we find the common vertices between face and adjacent face and their location
    common_vertices, common_vertices_index, _ = np.intersect1d(result_face, adjacent_face, return_indices=True)
    sorted_index = np.sort(common_vertices_index)
    # if there are only two common vertices = edge is shared
    if len(sorted_index) == 2:
        # figure out which vertex in the triangle is the one that needs to be added
        incoming_vertex = set(adjacent_face).difference(common_vertices).pop()
        # find location to insert the new vertex
        # if vertices are next to each other then insert the new vertex at the bigger index e.g. [1, 2, 0] and new vertex goes between 2 and 0 i.e. insert it at index 2
        if abs(sorted_index[0]-sorted_index[1]) == 1:
            insert_location = sorted_index[1]
        # if vertices are not next to each other i.e. at the edges of the array then insert the new vertex at the smaller index, which would be the bigger index in the circular array, e.g. [0, 1, 2] and new vertex goes between 2 and 0, so smaller number is bigger index i.e insert it at index 0.
        else:
            insert_location = sorted_index[0]
        # and insert it
        result_face.insert(insert_location, incoming_vertex)
        
    # edge case handling when all three vertices of the triangular face are already part of the polygonal face array
    # e.g. if the preceeding faces have encircled the face before encountering it
    # = two or three edges are shared
    else:
        # these three vertices will occur in sequence in the polygonal face array (difference sequence then triangular face array)
        # delete the vertex in the middle of this sequence to add the triangle i.e. deleting two edges and creating a new one
        # boxy_2 is the example .stl that has this problem

        # if both start and end index are there then the vertex to be deleted is the last one in the sort i.e. circular array
        # e.g. [10  4  2] are the vertices in the face array
        # [4, 5, 3, 0, 1, 8, 9, 10, 2] are the vertices of the polygonal array
        # [8 0 7] are the locations in the polygonal array, [0 7 8] are the sorted result, delete vertex at location 8
        if 0 in sorted_index and len(result_face)-1 in sorted_index:
            del result_face[sorted_index[2]]
        # otherwise the sequence is in the middle of the array and the index in the middle should be deleted  
        else:
            del result_face[sorted_index[1]]      
    return result_face

In [5]:
raw_vertices, raw_faces = igl.read_triangle_mesh("cube.stl")
vertices, faces, _ = igl.remove_duplicates(raw_vertices, raw_faces, 0.00001)
face_normals = igl.per_face_normals(vertices, faces, np.ones((1, 3)))

EPSILON = 1e-5

# create polygonal mesh from triangular mesh
# polygonal_faces, polygonal_face_normals= build_polyhedral_mesh(faces, face_normals)
